## LSTM 실습

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

In [2]:
tf.set_random_seed(333)

In [2]:
sentence = ('''이제 좀 구체적인 이야기를 해보죠. scikit-learn 0.18 버전에 다층 퍼셉트론(Multi-layer Perceptron)이 추가되었습니다.
향후 CNN, RNN 모델이 추가되거나 GPU지원이 될까요? 아니요. 이미 파이썬에서 사용할 수 있는 훌륭한 딥러닝 라이브러리들이 있고
이 분야는 매우 빠르게 변합니다. 우리가 케라스나 텐서플로를 따라잡아야 할 이유가 없습니다. GPU를 사용하는 데 필요한 패키지를 
설치하는 일이 여전히 큰 장벽이라서 GPU 지원은 추가하지 않기로 했습니다. 또 GPU를 지원한다는 것은 근본적으로 알고리즘들을 
밑바닥부터 다시 작성해야 한다는 뜻입니다. 다른 패키지를 사용해 쉽게 구현할 수 있는데, scikit-learn에서 굳이 이렇게 해야 할 강한 동기를
찾지 못했습니다.''')

In [3]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [4]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

In [5]:
X_data = []
Y_data = []

In [6]:
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    
    if i%100 == 0:
        print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    X_data.append(x)
    Y_data.append(y)

0 이제 좀 구체적인  -> 제 좀 구체적인 이
100 추가되거나 GPU지 -> 가되거나 GPU지원
200 할 이유가 없습니다 ->  이유가 없습니다.
300 고리즘들을 
밑바닥 -> 리즘들을 
밑바닥부


In [10]:
batch_size = len(X_data)

In [11]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

In [12]:
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)

Tensor("one_hot:0", shape=(?, 10, 153), dtype=float32)


In [13]:
lstm_cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
multi_cells = rnn.MultiRNNCell([lstm_cell] *3, state_is_tuple=True)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [14]:
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


In [15]:
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [16]:
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [17]:
weights = tf.ones([batch_size, sequence_length])

In [18]:
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: X_data, Y: Y_data})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
#         print(i, j, ''.join([char_set[t] for t in index]), l)

In [20]:
results = sess.run(outputs, feed_dict={X: X_data})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

  좀 구체적인 이야기를 해보죠. scikit-learn 0.18 버전에 다층 퍼셉트론(Multi-layer Perceptron)이 추가되었습니다.
향후 CNN, RNN 모델이 추가되거나 GPU지원이 될까요? 아니요. 이미 파이썬에서 사용할 수 있는 훌륭한 딥러닝 라이브러리들이 있고
이 분야는 매우 빠르게 변합니다. 우리가 케라스나 텐서플로를 따라잡아야 할 이유가 없습니다. GPU를 사용하는 데 필요한 패키지를 
설치하는 일이 여전히 큰 장벽이라서 GPU 지원은 추가하지 않기로 했습니다. 또 GPU를 지원한다는 것은 근본적으로 알고리즘들을 
밑바닥부터 다시 작성해야 한다는 뜻입니다. 다른 패키지를 사용해 쉽게 구현할 수 있는데, scikit-learn 서 굳이 이렇게 해야 할 강한 동기를
찾지 못했습니다.